# Template for Creating Sacrament Meeting Programs

## Uses MarkupPy to generate the HTML

In [19]:
#
# Sugar Land 2nd Ward HTML Program Writer
#
# Pete Slater
# June 2022

import hymndict # Titles and links to hymns
import htmlpy # Local copy of source code
from htmlpy import oneliner as e
import artlinks # Links to artwork
import sys
import datetime as dt

# Customized functions to create the HTML for standard agenda items

In [20]:
'''
Define functions for formatting each of the items that can appear in a program.
Build the program in the main script by calling the functions.

'''
def officer(role, officer, page):
    page.p(e.b(role+":")+" "+officer,align="center")

def pagetitle(unit, meeting, meetdate,page):
    page.h1(unit, align="center")
    page.h2(meeting, align="center")
    page.h2(meetdate, align="center")

def speaker(name, page):
    page.h2("Speaker", align="center")
    page.p(name,align="center")
    
def testimonies(page):
    page.h2("Bearing of Testimonies", align="center")

def music(number, description, page):
    page.h2(description,align="center")
    [hymntitle, hymnurl] = hymndict.hymns[number]
    page.p(e.a("#" + str(number)+", "+hymntitle, href=hymnurl),align="center")
    
def specialmusic(performers,page,title=None,accompanist=None):
    page.h2("Special Musical Number",align="center")
    if title != None:
        page.p(e.i(title),align="center")
    page.p(performers,align="center")
    if accompanist != None:
        page.p("Acc. by " + accompanist, align="center")
        
def thought(text, author, page):
    page.hr()
    page.p(text, align="center")
    page.p("- "+author,align="center")
 
# Announcements are passed as a list of text strings
def announcements(textlist,page):
    page.hr()
    page.h1("Announcements", align="center")
    for txt in textlist:
        page.p(txt,align="center")
        
# Calendar items are passed as a list of text strings
def calendar(textlist,page):
    page.hr()
    page.h1("Calendar Items", align="center")
    for txt in textlist:
        page.p(txt,align="center")

# Custom section for Preparedness Corner, etc.
def custom(title, txtlist, linklist, page):
    page.hr()
    page.h1(title, align="center")
    if txtlist != None:
        for txt in txtlist:
            page.p(txt,align="center")
    if linklist != None:
        for link in linklist:
            page.p(e.a(link[0], href=link[1], target="_blank", rel="noreferrer noopener"),align="center")
        
# Place some links, passed as a list containing text and url
def links(linklist, page):
    page.hr()
    page.h1("Links", align="center")
    for link in linklist:
        page.p(e.a(link[0], href=link[1], target="_blank", rel="noreferrer noopener"),align="center")
        
# Print credits at the bottom of the page
def credit(editor, page):
#'''Print credits and datestamp ''' 
    today = dt.date.today()
    page.hr()
    text = today.strftime("Edited %B %d, %Y by ") + editor
    page.small(text,align="center")        
    

## Functions for the cleaning calendar

In [21]:
def nextsunday(n):
#""" Return next n Sundays, including today if it's Sunday"""
 today = dt.date.today()
 daynum = today.weekday() # Sunday is day #6

 if daynum == 6:
  basedate = today 
 else:
  basedate = today + dt.timedelta(6-daynum)
    
 sundays = []
 for i in range(n):
  nxtSunday = basedate + dt.timedelta(i*7)
  sundays.append(nxtSunday.strftime("%B %d, %Y"))
 return sundays

def nextsaturday(n):
#""" Return next n Saturdays, including today if it's Saturday"""
 today = dt.date.today()
 daynum = today.weekday() # Saturday is day #5

 if daynum == 5:
  basedate = today 
 else:
  basedate = today + dt.timedelta(5-daynum)
    
 saturdays = []
 for i in range(n):
  nxtSaturday = basedate + dt.timedelta(i*7)
  saturdays.append(nxtSaturday.strftime("%B %d"))
 return saturdays

In [22]:
roster = {
    "June 24" : "Sugar Land 1st",
    "July 01" : "Bradford, Arnold, Emerson, Markwalter",
    "July 08" : "Clark, Lambert, Blackburn, Ganci, Martin, Robinson",
    "July 15" : "Crandall, Blanco, Garcia, Moline, Thibault",
    "July 22" : "Crosby, Boyer, Griffin, Mike & Dawnette Moore, MacKinnon",
    "July 29" : "Maplewood 3rd Branch",
    "August 05" : "Draney, Buchanan, Hettinger, Morton, Tracey",
    "August 12" : "Leavitt, Chen, Hintze, Ozomah, Seegmiller",
    "August 19" : "Powell, Clements, Horowitz, Quam, Wightman",
    "August 26" : "Kaye Reynolds, Debow, Jeffery, Rasmussen, Winningham",
    "September 02" : "Roskelley, Ekstrom, Ocampo, Juarez, Jason & Deann Moore, Slater",
    "September 09" : "Arnold, Bradford, Emerson, Markwalter",
    "September 16" : "Blackburn, Clark, Ganci, Lambert, Martin, Robinson",
    "September 23" : "Blanco, Crandall, Garcia, Moline, Thibault",
    "September 30" : "Maplewood 3rd Branch",
    "October 07" : "Boyer, Crosby, Griffin, MacKinnon, Mike & Dawnette Moore",
    "October 14" : "Buchanan, Draney, Hettinger, Morton, Tracey",
    "October 21" : "Chen, Hintze, Leavitt, Ozomah, Seegmiller",
    "October 28" : "Clements, Horowitz, Powell, Quam, Wightman",
    "November 04" : "Debow, Jeffery, Rasmussen, Kay Reynolds, Winningham",
    "November 11" : "Ekstrom, Juarez, Jason & Deann Moore, Ocampo, Roskelley, Slater",
    "November 18" : "Arnold, Bradford, Emerson, Markwalter",
    "November 25" : "Blackburn, Clark, Ganci, Lambert, Martin, Robinson",
    "December 02" : "Blanco, Crandall, Garcia, Moline, Thibault",
    "December 09" : "Boyer, Crosby, Griffin, MacKinnon, Mike & Dawnette Moore",
    "December 16" : "Buchanan, Draney, Hettinger, Morton, Tracey",
    "December 23" : "Chen, Hintze, Leavitt, Ozomah, Seegmiller",
    "December 30" : "Maplewood 3rd Branch",
    "January 06" : "Clements, Horowitz, Powell, Quam, Wightman",
    "January 13" : "Debow, Jeffery, Rasmussen, Kay Reynolds, Winningham",
    "January 20" : "Ekstrom, Juarez, Jason & Deann Moore, Ocampo, Roskelley, Slater",
}

# Format the agenda items that are always present
Customize for each week with date and type of meeting, i.e. sacrament, fast and testimony, ward conference

In [23]:
Title = "Sugar Land Second Ward"
Title = "Houston Texas South Stake"
#header = "Sacrament Meeting"
#header = "Fast and Testimony Meeting"
header = "Stake Conference"
#header = "General Conference"
footer = ""
styles = ( 'layout.css', 'alt.css', 'images.css' )

page = htmlpy.page( )
page.init()
page.br( )
 
# Make sure it will look good on all devices
page.meta(name="viewport", content="width=device-width, initial-scale=1.0")

# Define the elements on the current week's programs here

#pagetitle("Sugar Land Second Ward", "General Conference", "October 1st, 2023", page)
pagetitle("Sugar Land Second Ward", "Sacrament Meeting", "December 10, 2023", page)
#pagetitle("Sugar Land Second Ward", "Stake Conference", "December 2-3, 2023", page)

## Artwork
Pick an artwork from the defined list of links in artlinks.py and enter the number here

Christmas-themed keys are "Nativity1", "Annunc", "Birth", "Simon"

In [24]:
# Place an artwork from the imported dictionary of links
#page.p(e.img(width=299, height=300*0.8, src=artlinks.art[2]), align="center")
page.p(e.img(style="max-width:50%;height:auto;", src=artlinks.art['Birth']), align="center")

## Officers

In [25]:
officer("Presiding","Bishop Joey Powell", page)
#officer("Conducting","Bishop Joey Powell", page)
#officer("Presiding","Brother Brent Leavitt", page)
#officer("Conducting","Brother Jared Draney", page)
#officer("Presiding","Brother Brent Leavitt", page)
officer("Conducting","Brother Brent Leavitt", page)
#officer("Presiding","President Travis Bird", page)

## Opening and sacrament hymns

In [26]:
#music(212,"Opening Hymn",page)
#music(188,"Sacrament Hymn", page) 

## Variable part of the program

### If it's fast and testimony meeting, uncomment this section and comment out the sacrament meeting items

In [27]:
# Uncomment this line for fast and testimony meeting
#testimonies(page)

### For a sacrament meeting or ward conference, comment out the testimonies and use these sections as needed

In [28]:
# Use lines like these to form the speaking part of the program

# Special code for the Primary and Christmas Programs
#txtlist = (
#    'Words and Music Celebrating the Birth of Jesus Christ',
#    'Program under the direction of Mindy Draney',
#    'Program written by Becky Slater',
#    'Congregation to join in singing when directed.'
#    )
#linklist = None

#custom("Primary Program", txtlist,linklist, page)
#specialmusic("Primary Children",page, title="Fathers")
#specialmusic("Primary Children",page, title= "Primary Program - I'm trying to be like Jesus")

#specialmusic("Sister Strong and Sister Allen",page, title= "I Need Thee Every Hour")

#speaker("Sister Avery Allen", page)

#speaker("Sister Kenia Gutierrez", page)

speaker("Tracy Horowitz", page)

#music(98,"Rest Hymn",page)

speaker("Ruth Jeffery", page)

In [29]:
## Stake Conference Code - usually commented out

In [30]:
# # Special Code for Stake Conference
# txtlist = (
#     "Bishopric, Ward Council, Counselors, Secretaries, and those who attend missionary and temple and family history correlation meetings",
#     "In-person at Lexington"
# )
# linklist = None

# custom("Leadership Session, Saturday 2:00-4:30 pm", txtlist, linklist, page)

# txtlist = (
#     "All Adults",
#     "In-person at Lexington"
# )
# linklist = None

# custom("Adult Session, Saturday 7:00-8:30 pm", txtlist, linklist, page)

# txtlist = (
#     "In-person at Lexington",
#     "Via Zoom at Sienna and Rosenberg"
# )
# linklist = None

# #custom("General Session, Sunday 10:00 am - 12:00 pm", txtlist, linklist, page)

### Closing hymn

In [31]:
#music(209,"Closing Hymn",page)

## End matter - thought, announcements, calendar, links

In [32]:
# Spiritual thought
quote = ('Deviating to the right or the left of the safe track ahead of us, whether because of laziness or rebelliousness, can prove fatal to our spiritual lives. There are no exceptions to this rule.')
author = 'Elder Patrick Kearon'
thought(quote, author, page)

# Make a list of announcements, then post to the page
txtlist = (
        "Sacrament meetings will begin at 9:00 AM in 2024", 
        "Tithing Declaration is ongoing through December 17th. Please see sign-up sheet on the bishop's office.",
        "The Houston Texas South Mission seeks senior couples to work in the office",
        "The Angel Tree sponsored by the YM/YW is in the foyer for a couple more weeks",
        "Family Search training classes will start in January. Email tomkrouskop@att.net for more information.",
        "Volley Ball Wednesdays - Come join us on Wednesdays at 9PM and have fun!!!",
        "Send announcements to bvl2clerk@gmail.com")
announcements(txtlist, page)

callist = (
    "December 10 - Houston South Stake Christmas Concert at Lexington Building, 6:00 pm",
    "December 10 - Senior Missionary Fireside, 7:00 pm at The Woodlands Stake Center",
    "December 15 - Houston South Stake Single Adults Christmas Lights Hayride, 6:30 PM - 8:30PM",
    "January 15 - MLK Day of Service. Volunteers needed",
    "January 27 - Family History Fair at the Richmond Stake Center 9:00 am - 2:00 pm",
    "March 13-16 - Stake Youth Trek",       
           )
calendar(callist, page)


# Cleaning Assignments

In [33]:
sats = nextsaturday(4)
cleaninglist = []
for sat in sats:
    cleaninglist.append(sat + ": " + roster[sat])
custom("Cleaning Assignments",cleaninglist, None, page)

### Preparedness Corner - under development

In [34]:
txtlist = (
"Emergency preparedness is the process of planning and preparing for potential hazards and disasters. Even though hurricane season for the Houston area is pretty much over, emergencies don't pay attention to the calendar.",
"This December is as good a time as any to increase your family's preparedness. If you don't have 72-hour kits packed for everyone in your household (even if it's just you), let this guide you in your Christmas shopping!",
"If you're all prepared in that sense, now is a good time to wrap your outside faucets and do all else that you need to in order to be prepared for any upcoming freezes.",
"There are countless lists and resources online to give guidance and practical information on emergency preparedness. Go ahead and get started!",
    )
#linklist = {
#    ("Learn more online","https://www.churchofjesuschrist.org/study/manual/gospel-topics/food-storage?lang=eng")
#}
linklist = None
custom("Houston South Stake Emergency Preparedness Corner", txtlist,linklist, page)

In [35]:
# Make a list of links, then post
linklist = (
        ("Let the missionaries know when you are available to help teach","https://docs.google.com/forms/d/1cVhEINFFESai0MLbgBA8euTyiMQsM8R1Sur_EQsyUrw/edit?usp=drive_web"),
        ("See to Succeed a Houston South Stake service project","https://www.houstonhealth.org/services/clinical/see-to-succeed"),
        ("Sign up to feed the missionaries","https://www.signupgenius.com/go/10c0e4baca82ea7fb6-dinner2"),
        ("Sign up to prepare a meal with Fort Bend Family Promise","https://www.signupgenius.com/go/10c0e4baca82ea7fb6-helping4#"),
        ("Help at secondmile.org","https://secondmile.org"), 
        )
links(linklist, page)
credit("Pete Slater and Andy Chen", page)

# Write out the file to Index.html, which must be posted online

In [36]:
print (page)# -*- coding: utf-8 -*

original_stdout = sys.stdout
with open('Index.html', 'w') as f:
    sys.stdout = f # Change the standard output to the file we created.
    print(page)
    sys.stdout = original_stdout # Reset the standard output to its original value
"""
End of script
"""

<!DOCTYPE HTML PUBLIC '-//W3C//DTD HTML 4.01 Transitional//EN'>
<html lang="en">
<head>
</head>
<body>
<br />
<meta name="viewport" content="width=device-width, initial-scale=1.0" />
<h1 align="center">Sugar Land Second Ward</h1>
<h2 align="center">Sacrament Meeting</h2>
<h2 align="center">December 10, 2023</h2>
<p align="center"><img style="max-width:50%;height:auto;" src="https://www.lds.org/bc/content/shared/content/images/gospel-library/magazine/ensignlp.nfo:o:19b.jpg" /></p>
<p align="center"><b>Presiding:</b> Bishop Joey Powell</p>
<p align="center"><b>Conducting:</b> Brother Brent Leavitt</p>
<h2 align="center">Speaker</h2>
<p align="center">Tracy Horowitz</p>
<h2 align="center">Speaker</h2>
<p align="center">Ruth Jeffery</p>
<hr />
<p align="center">Deviating to the right or the left of the safe track ahead of us, whether because of laziness or rebelliousness, can prove fatal to our spiritual lives. There are no exceptions to this rule.</p>
<p align="center">- Elder Patrick Kea

'\nEnd of script\n'